<a href="https://colab.research.google.com/github/Matheus0820/Introducao-a-Otimizacao/blob/main/Problema_da_distribui%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema da distribuição

## Descrição do Problema

Você foi contratado por uma empresa do ramo de bebidas para resolver o problema que é descrito a seguir, tendo em vista a proximidade do feriado de carnaval.

A empresa possui $m$ fábricas, $n$ centros de distribuição e $p$ clientes que devem ser atendidos. As fábricas, centros de distribuição e clientes estão localizados em diversos locais pelo Brasil. Portanto, o custo de transporte de produtos entre fábricas e centros, bem como centros e clientes, pode variar de acordo com a distância, pedágios ou até mesmo o modal que é utilizado para fazer este transporte.

Considerando um produto específico, a empresa estima que no feriado de carnaval cada cliente $k~(k=1, 2, ..., p)$ terá uma demanda específica de $d_k$ unidades por este produto.

A empresa estima também que produzir uma unidade do produto na fábrica $i$ e transportá-la para o centro de distribuição $j$ tem um custo $c_{ij}$, para todo $i=1, 2, ..., m$ e para todo $j=1, 2, ..., n$. De maneira similar, armazenar e transportar uma unidade do produto do centro de distribuição $j$ para o cliente $k$ apresenta um custo $t_{jk}$, para todo $j=1, 2, ..., n$ e para todo $k=1,2, ...,p$.

Em relação ao processo de **produção**, sabe-se que uma fábrica $i$ tem um custo fixo de produção $f_i$. Isto é, se ela é usada para produzir uma ou mais unidades do produto, há um custo a ser pago, relacionado à configuração da linha de produção, funcionários, dentre outros recursos operacionais que serão disponibilizados. Cada uma destas fábricas possui também uma capacidade de produção, representada por $g_i$.

Em relação ao processo de **distruibuição**, sabe-se que um centro $j$ tem um custo fixo de operação $h_j$. Isto é, se ele é usado armazenar e transportar uma ou mais unidades do produto, há um custo a ser pago, relacionado à configuração do centro, como alocação dos funcionários, destinação do espaço, energia elétrica, dentre outros recursos. Cada um destes centros possui também uma capacidade de armazenando e processamento, representado por $q_j$.

## Carregando depedências do AMPL


In [ ]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="79c45049-0fee-47f7-a329-ec026ff58c82") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 34.0 MB/s eta 0:00:00
Licensed to AMPL Academic Community Edition License for <matheus.ramos.703@ufrn.edu.br>.


## Paramêtros:
- $m$: Quantidade de fábricas;
- $n$: Quantidade de centros de distribuição;
- $p$: Quantidade de clientes;
- $d_k$: Demanda de clientes $k$ por unidade de certo produto;
- $c_{ij}$: Custo de produto na **fabrica** $i$ e transporte para **distribuidora** $j$;
- $t_{jk}$: Custo de transporte de uma unidade da **distribuidora** $j$ para um **cliente** $p$;
- $f_i$: Custo de produção da **fabrica** $i$;
- $g_i$: Capacidade de produção da **fabrica** $i$;
- $h_j$: Custo de operação da **distribuidora** $j$;
- $q_j$: Capacidade da **distribuidora** $j$;

## Variáveis:
- $x_{ij} \in \mathbb{R}^+$ : Quantidade de produtos levado da fabrica $i$ para distribuidora $j$

- $y_{jk} \in \mathbb{R}^+$ : Quantidade de produtos levados da distribuidora $j$ para o cliente $k$

- $x'_i$ = $\begin{cases}
      \mbox{1, se Fabrica i é usada} \\
      \mbox{0, caso contrário}
  \end{cases}$

- $y'_j$ = $\begin{cases}
      \mbox{1, se o centro de distribuição j é usada} \\
      \mbox{0, caso contrário}
  \end{cases}$

## Função objetivo:

$$
\mbox{min Z} = \sum_{i = 1}^{m} \left[ f_i x'_i + \sum_{j = 1}^{n} c_{ij}x_{ij} \right] + \sum_{j = 1}^{n} \left[ h_j y'_j + \sum_{k = 1}^{p} t_{jk}y_{jk} \right] \tag{1}
$$

## Sujeito a:
$$
\sum_{j = 1}^{n}x_{ij} \leq g_ix'_i, ∀ i = 1..m \tag{2}
$$

$$
\sum_{k = 1}^{p} y_{jk} \leq q_jy'_j, ∀ j = 1..n \tag{3}
$$

$$
\sum_{i = 1}^{m} \sum_{j = 1}^{n} x_{ij} = \sum_{k = 1}^{p} d_k \tag{4}  
$$


## Arquivo dados.dat

In [ ]:
%%writefile dados.dat
param m := 2;
param n := 3;
param p := 3;
param d :=
1 80
2 90
3 40;
param c :	 1	2	 3 :=
1	10	8	20
2	20	6	11;
param t :	 1	2	 3 :=
1	 7	5	10
2	 5	4	3
3	10	8	6;

param f :=
1 300
2 250;
param g :=
1 120
2 100;
param h :=
1 60
2 80
3 100;
param q :=
1 70
2 90
3 66;


Writing dados.dat


## Arquivo modelo.mod

In [ ]:
%%writefile modelo.mod
param m >= 1;
param n >= 1;
param p >= 1;
param d {1..p};
param c {1..m, 1..n};
param t {1..n, 1..p};
param f {1..m};
param g {1..m};
param h {1..n};
param q {1..n};

var x {1..m, 1..n} >= 0;
var y {1..n, 1..p} >= 0;
var x_ {1..m} binary;
var y_ {1..n} binary;

minimize Z: sum{i in 1..m} (f[i]*x_[i] + sum{j in 1..n} (c[i,j]*x[i,j])) + sum{j in 1..n} (h[j]*y_[j] + sum{k in 1..p} (t[j,k]*y[j,k]));

subject to capacidade_fabrica {i in 1..m}: sum{j in 1..n} x[i,j] <= g[i]*x_[i];
subject to capacidade_transportadora {j in 1..n}: sum{k in 1..p} y[j,k] <= q[j]*y_[j];
subject to demanda: sum{i in 1..m, j in 1..n} x[i,j] == sum{k in 1..p} d[k];

Overwriting modelo.mod


## Cofingurando e execultando o algoritimo
> Usando a tag ***%%ampl_eval***

In [ ]:
%%ampl_eval
reset;
model modelo.mod;
data dados.dat;
option solver cplex;
solve;
display Z;

CPLEX 22.1.2: CPLEX 22.1.2: optimal solution; objective 2030
0 simplex iterations
Z = 2030

